In [1]:
from snowflake.snowpark import Session
import warnings; warnings.simplefilter('ignore')
import configparser

In [2]:
# Fosfor ML
from fosforml import *
from fosforml.constants import MLModelFlavours
import requests

In [10]:
import pandas as pd
import numpy as np

In [3]:
#Import all snowflake connection details from template variables.
db_user = 'ASHUTOSH'
db_password = 'Password@2023'
db_account = 'ug94937.us-east4.gcp'
db_database =  'RETAIL_DB'
db_role = 'ASHUTOSH'
db_warehouse = 'FOSFOR_SOLUTIONS_WH'
db_schema = 'CPG_SAP_OTIF_SCHEMA'

In [4]:
from snowflake.snowpark.session import Session
connection_params = {
    'user': db_user,
    'password': db_password,
    'account': db_account,
    'warehouse': db_warehouse,
    'database': db_database,
    'schema': db_schema,
    'role': db_role
}
session1 = Session.builder.configs(connection_params).create()

In [5]:
df = session1.table("ENRICHED_ORDER_SHIP")

In [6]:
df = df.to_pandas()

In [7]:
df.shape

(966997, 46)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966997 entries, 0 to 966996
Data columns (total 46 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   CUSTOMER_PURCHASE_ORDER_DATE          966997 non-null  object 
 1   ORDER_ENTRY_DATE                      966997 non-null  object 
 2   ORDER_REQUIRED_DELIVERY_DATE          966997 non-null  object 
 3   ORDER_CUSTOMER_REQUEST_DELIVERY_DATE  966997 non-null  object 
 4   ORDER_PROMISE_DATE                    966997 non-null  object 
 5   ORDER_FIRST_SHIPMENT_DATE             966997 non-null  object 
 6   SHIPMENT_ACTUAL_START_DATE            966997 non-null  object 
 7   ORDER_LINECANCEL_DATE                 966997 non-null  object 
 8   ITEM_OTQF                             758063 non-null  object 
 9   ITEM_OTQF_FLAG                        966997 non-null  object 
 10  ITEM_OTQF_DAYS                        847138 non-null  float64
 11  

In [16]:
#Dropping all the ID related columns

cols_to_drop = ['ITEM_OTQF','ITEM_ORDERFILL','ITEM_CUSTOMER','ORDER_NUMBER']
df.drop(cols_to_drop, inplace=True, axis=1)

In [17]:
df.shape

(966997, 42)

In [18]:
cat_columns = ['CUSTOMER_NAME','SHIPMENT_ORIGINATION_FACILITY','PRODUCT_TYPE','PACKAGE_TYPE','PRODUCT_TECHNOLOGY',
'PRODUCT_SUBTECHNOLOGY','PRODUCT_DESIGN_TYPE','PRODUCT_SIZE','SUB_BRAND','CITY','OUTLETS','REGION','COUNTRY','CHANNEL',
'SHIPMENT_ORIGINATION_FACILITY_TYPE','BRAND']

for col in cat_columns:
    print ("------------------------------------------------")
    print (col, " -> ",df[col].nunique())

------------------------------------------------
CUSTOMER_NAME  ->  80
------------------------------------------------
SHIPMENT_ORIGINATION_FACILITY  ->  17
------------------------------------------------
PRODUCT_TYPE  ->  1
------------------------------------------------
PACKAGE_TYPE  ->  3
------------------------------------------------
PRODUCT_TECHNOLOGY  ->  1
------------------------------------------------
PRODUCT_SUBTECHNOLOGY  ->  3
------------------------------------------------
PRODUCT_DESIGN_TYPE  ->  3
------------------------------------------------
PRODUCT_SIZE  ->  7
------------------------------------------------
SUB_BRAND  ->  32
------------------------------------------------
CITY  ->  3369
------------------------------------------------
OUTLETS  ->  7
------------------------------------------------
REGION  ->  67
------------------------------------------------
COUNTRY  ->  6
------------------------------------------------
CHANNEL  ->  3
-------------------

In [19]:
df[cat_columns].head()

,CUSTOMER_NAME,SHIPMENT_ORIGINATION_FACILITY,PRODUCT_TYPE,PACKAGE_TYPE,PRODUCT_TECHNOLOGY,PRODUCT_SUBTECHNOLOGY,PRODUCT_DESIGN_TYPE,PRODUCT_SIZE,SUB_BRAND,CITY,OUTLETS,REGION,COUNTRY,CHANNEL,SHIPMENT_ORIGINATION_FACILITY_TYPE,BRAND
0,WALMART - US,Yuma Plant,Beverage,Regular,Soft Drinks,Aerated Drinks,Cans,12 Oz,Craisins Trail,El Paso,Hyperstore,TX,United States,Modern Channel,Distribution center,Craisins Trail
1,WALMART - US,Fort Wayne DC,Beverage,Regular,Soft Drinks,Aerated Drinks,Cans,12 Oz,Craisins Trail,Somerset,Hyperstore,KY,United States,Modern Channel,Distribution center,Craisins Trail
2,WALMART - US,Fort Wayne DC,Beverage,Regular,Soft Drinks,Aerated Drinks,Cans,12 Oz,Craisins Trail,Ludington,Hyperstore,MI,United States,Modern Channel,Distribution center,Craisins Trail
3,WALMART - US,San Antonio 3PL,Beverage,Regular,Soft Drinks,Energy Drink,Cans,16 Oz,Craisins Trail,La Grange,Hyperstore,TX,United States,Modern Channel,3PL Third Party Logistics,Craisins Trail
4,WALMART - US,San Antonio 3PL,Beverage,Regular,Soft Drinks,Aerated Drinks,Cans,12 Oz,Craisins Trail,Uvalde,Hyperstore,TX,United States,Modern Channel,3PL Third Party Logistics,Craisins Trail


In [21]:
# Drop categorical columns
cat_cols_to_drop = ['PRODUCT_TYPE','PRODUCT_TECHNOLOGY','CITY','SUB_BRAND']
df.drop(cat_cols_to_drop, inplace=True, axis=1)

In [22]:
df.shape

(966997, 38)

In [23]:
date_columns = [
'CUSTOMER_PURCHASE_ORDER_DATE','ORDER_ENTRY_DATE','ORDER_REQUIRED_DELIVERY_DATE','ORDER_CUSTOMER_REQUEST_DELIVERY_DATE',
    'ORDER_PROMISE_DATE','ORDER_FIRST_SHIPMENT_DATE','SHIPMENT_ACTUAL_START_DATE','ORDER_LINECANCEL_DATE'
]

In [27]:
for col in date_columns:
    df[col] = pd.to_datetime(df[col])

In [35]:
df['CUSTOMER_ORDER_YEAR'] = df['CUSTOMER_PURCHASE_ORDER_DATE'].dt.year
df['CUSTOMER_ORDER_MONTH'] = df['CUSTOMER_PURCHASE_ORDER_DATE'].dt.month
df['CUSTOMER_ORDER_WEEK'] = df['CUSTOMER_PURCHASE_ORDER_DATE'].dt.weekday

In [40]:
df['CUSTOMER_ORDER_ENTRY_IN_DAYS'] = (df['CUSTOMER_PURCHASE_ORDER_DATE'] - df['ORDER_ENTRY_DATE']).dt.days
df['CUSTOMER_ORDER_REQUIRE_DELV_IN_DAYS'] = (df['CUSTOMER_PURCHASE_ORDER_DATE'] - df['ORDER_REQUIRED_DELIVERY_DATE']).dt.days
df['CUSTOMER_ORDER_REQ_DELV_IN_DAYS'] = (df['CUSTOMER_PURCHASE_ORDER_DATE'] - df['ORDER_CUSTOMER_REQUEST_DELIVERY_DATE']).dt.days
df['CUSTOMER_ORDER_PROM_DELV_IN_DAYS'] = (df['CUSTOMER_PURCHASE_ORDER_DATE'] - df['ORDER_PROMISE_DATE']).dt.days

df['ORDER_ENTRY_PROMISE_IN_DAYS'] = (df['ORDER_ENTRY_DATE'] - df['ORDER_PROMISE_DATE']).dt.days
df['ORDER_ENTRY_SHIPMT_START_IN_DAYS'] = (df['ORDER_ENTRY_DATE'] - df['SHIPMENT_ACTUAL_START_DATE']).dt.days

In [42]:
df.drop(date_columns, inplace=True, axis=1)

In [43]:
df.shape

(966997, 41)

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966997 entries, 0 to 966996
Data columns (total 41 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   ITEM_OTQF_FLAG                       966997 non-null  object 
 1   ITEM_OTQF_DAYS                       847138 non-null  float64
 2   ITEM_ORDERFILL_FLAG                  966997 non-null  object 
 3   ITEM_ORDERFILL_DAYS                  957859 non-null  float64
 4   ITEN_CUSTOMER_ORDERFILL_FLAG         966997 non-null  int8   
 5   ITEM_CUSTOER_DAYS                    847138 non-null  float64
 6   OPEN_ORDER_FLAG                      966997 non-null  object 
 7   BACK_ORDER_FLAG                      966997 non-null  object 
 8   CANCEL_FLAG                          966997 non-null  object 
 9   CUSTOMER_NAME                        966997 non-null  object 
 10  SHIPMENT_ORIGINATION_FACILITY        966997 non-null  object 
 11  PACKAGE_TYPE 

In [44]:
df.isnull().sum()

ITEM_OTQF_FLAG                              0
ITEM_OTQF_DAYS                         119859
ITEM_ORDERFILL_FLAG                         0
ITEM_ORDERFILL_DAYS                      9138
ITEN_CUSTOMER_ORDERFILL_FLAG                0
ITEM_CUSTOER_DAYS                      119859
OPEN_ORDER_FLAG                             0
BACK_ORDER_FLAG                             0
CANCEL_FLAG                                 0
CUSTOMER_NAME                               0
SHIPMENT_ORIGINATION_FACILITY               0
PACKAGE_TYPE                                0
PRODUCT_SUBTECHNOLOGY                       0
PRODUCT_DESIGN_TYPE                         0
PRODUCT_SIZE                                0
OUTLETS                                     0
ORDER_QUANTITY                              0
QUANTITY_SHIPPED_ORDERFILL                  0
QUANTITY_SHIPPED_CUSTOMER                   0
QUANTITY_SHIPPED_OTQF                       0
DELIVERED_QUANTITY                          0
DELIVERED_QUANTITY_OTQF           

In [15]:
df[['ITEM_OTQF','ITEM_OTQF_DAYS','ITEM_ORDERFILL','ITEM_ORDERFILL_DAYS','ITEM_CUSTOMER','ITEM_CUSTOER_DAYS']].head()

,ITEM_OTQF,ITEM_OTQF_DAYS,ITEM_ORDERFILL,ITEM_ORDERFILL_DAYS,ITEM_CUSTOMER,ITEM_CUSTOER_DAYS
0,1100971703_110,2.0,1100971703_110,2.0,None,2.0
1,1100971621_60,5.0,1100971621_60,5.0,1100971621_60,3.0
2,1100971689_120,3.0,1100971689_120,3.0,1100971689_120,1.0
3,1100971592_10,3.0,1100971592_10,3.0,1100971592_10,3.0
4,1100971623_50,4.0,1100971623_50,4.0,1100971623_50,4.0


In [11]:
num_df = df.select_dtypes(include=[np.number])

In [12]:
num_df

,ITEM_OTQF_DAYS,ITEM_ORDERFILL_DAYS,ITEN_CUSTOMER_ORDERFILL_FLAG,ITEM_CUSTOER_DAYS,ORDER_NUMBER,ORDER_QUANTITY,QUANTITY_SHIPPED_ORDERFILL,QUANTITY_SHIPPED_CUSTOMER,QUANTITY_SHIPPED_OTQF,DELIVERED_QUANTITY,DELIVERED_QUANTITY_OTQF,OPEN_QUANTITY,BACK_ORDER_QUANTITY,CANCEL_QUANTITY
0,2.0,2.0,0,2.0,1100971703,3,3,0,3,3,3,0,0,0
1,5.0,5.0,1,3.0,1100971621,1,1,1,1,1,1,0,0,0
2,3.0,3.0,1,1.0,1100971689,2,2,2,2,2,2,0,0,0
3,3.0,3.0,1,3.0,1100971592,1,1,1,1,1,1,0,0,0
4,4.0,4.0,1,4.0,1100971623,12,12,12,12,12,12,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
966992,-5.0,-5.0,0,-5.0,1101157585,18,0,0,0,18,0,0,0,0
966993,5.0,5.0,1,5.0,91979039,2,2,2,2,2,2,0,0,0
966994,3.0,3.0,1,3.0,91979130,2,2,2,2,2,2,0,0,0
966995,NaN,2.0,0,NaN,91979194,2,0,0,0,0,0,0,0,2
